# Recitation 6
Topics: Higher order functions

## Exercise: Fold

Implement your own `foldLeft` function using:
1. A loop
2. Recursion
3. Inference rules

In [1]:
def fold_with_loop(l: List[Int], init_acc: Int, f: (Int, Int) => Int): Int = {
    var acc = init_acc
    for (element <- l) {
        acc = f(acc, element)
    }
    acc
}

val test_list = List(1, 5, 9)
assert(fold_with_loop(test_list, 0, _ + _) == 15)

defined function fold_with_loop
test_list: List[Int] = List(1, 5, 9)

In [2]:
def fold_with_recursion(l: List[Int], init_acc: Int, f: (Int, Int) => Int): Int = 
    l match {
        case Nil => init_acc
        case element :: rest => fold_with_recursion(rest, f(init_acc, element), f)
    }

assert(fold_with_recursion(test_list, 0, _ + _) == 15)

defined function fold_with_recursion

Nil rule:
$$
\newcommand{\t}[1]{\texttt{#1}}
\begin{array}{c}
\hline
\t{fold}(\t{Nil},\ init\_acc,\ f) = init\_acc \\
\end{array}
$$

---

Cons rule:
$$
\begin{array}{c}
f(init\_acc,\ element) = n' \\
\t{fold}(rest,\ n',\ f) = n'' \\
\hline
\t{fold}(element :: rest,\ init\_acc,\ f) = n'' \\
\end{array}
$$

## Exercise: Translate
Translate the recursive function to a fold, then to inference rules

In [3]:
def map_rec(l: List[Int], f: Int => Int): List[Int] = {
    l match {
        case Nil => Nil
        case element :: rest => f(element) :: map_rec(rest, f)
    }
}

// Implement map with fold
def map_fold(l: List[Int], f: Int => Int): List[Int] =
    l.foldRight(Nil: List[Int])(
        (element, acc) => f(element) :: acc
    )

val f: Int => Int = _ + 1
val l = List(3, 94, 0)
assert(map_fold(l, f) == l.map(f))

defined function map_rec
defined function map_fold
f: Int => Int = ammonite.$sess.cmd2$Helper$$Lambda$2692/0x0000000800d09840@2adc8ed4
l: List[Int] = List(3, 94, 0)

Nil rule:
$$
\newcommand{\t}[1]{\texttt{#1}}
\begin{array}{c}
\hline
\t{map}(\t{Nil},\ f) = \t{Nil} \\
\end{array}
$$

---

Cons rule:
$$
\begin{array}{c}
f(\ element) = element' \\
\t{map}(rest, \ f) = rest' \\
\hline
\t{map}(element :: rest,\ f) = element' :: rest' \\
\end{array}
$$

## Exercise: Manipulating logs

We have a file that contains a series of log messages, and we want to manipulate them programatically.

In [4]:
val logs = List(
    "ERROR   (0034) Your toast is burning",
    "WARNING (0030) No need for a good error message, this'll never break!",
    "WARNING (0031) Stack overflow is down, abandon ship!",
    "ERROR   (0026) 'cpu' not found. To continue, please run `apt install cpu`",
    "ERROR   (0032) Perfectly balanced...",
    "WARNING (0033) ... as all things should be",
    "ERROR   (0028) \"'You miss 100% of the shots you don't take -Wayne Gretsky' -Michael Scott\" -PPL",
    "ERRORR  (ABCD) Error: there are an errors in this error message.",
    "ERROR   (0027) The real slim shady didn't stand up",
    "ERROR   (5059) Send me $100 in bitcoin - from your boss",
    "WARNING (0029) Your computer is about to explode"
)

sealed trait AlertLevel
case object ERROR extends AlertLevel
case object WARNING extends AlertLevel

case class Log(val alert_level: AlertLevel, val time: Int, val msg: String)

// As an example, the first log in the list should parse to:
Log(ERROR, 34, "Your toast is burning")

logs: List[String] = List(
  "ERROR   (0034) Your toast is burning",
  "WARNING (0030) No need for a good error message, this'll never break!",
  "WARNING (0031) Stack overflow is down, abandon ship!",
  "ERROR   (0026) 'cpu' not found. To continue, please run `apt install cpu`",
  "ERROR   (0032) Perfectly balanced...",
  "WARNING (0033) ... as all things should be",
  "ERROR   (0028) \"'You miss 100% of the shots you don't take -Wayne Gretsky' -Michael Scott\" -PPL",
  "ERRORR  (ABCD) Error: there are an errors in this error message.",
  "ERROR   (0027) The real slim shady didn't stand up",
  "ERROR   (5059) Send me $100 in bitcoin - from your boss",
  "WARNING (0029) Your computer is about to explode"
)
defined trait AlertLevel
defined object ERROR
defined object WARNING
defined class Log
res3_5: Log = Log(ERROR, 34, "Your toast is burning")

In [5]:
// Setup for later. You can ignore this.
// This will download stuff the first time you run it

import $ivy.`org.scala-lang.modules::scala-parser-combinators:1.1.1`
import scala.util.parsing.combinator.RegexParsers

object LogParser extends RegexParsers {
    val id: Parser[Int] = raw"[0-9]{4}".r ^^ (_.toInt)
    def alert_level: Parser[AlertLevel] = ("WARNING" ^^^ WARNING) | ("ERROR" ^^^ ERROR)
    def msg: Parser[String] = raw".*".r
    def log: Parser[Log] = alert_level ~ ("(" ~> id <~ ")") ~ msg ^^ {
        case level ~ id ~ msg => Log(level, id, msg)
    }
    def apply(input: String) = parseAll(log, input)
}

import $ivy.$                                                       

import scala.util.parsing.combinator.RegexParsers


defined object LogParser

In [6]:
// More setup that can be ignored

import LogParser.{ ParseResult, Success, Failure }

def filter(result: ParseResult[Log], f: Log => Boolean, error_msg: String): ParseResult[Log] =
    result.filterWithError(f, (_) => error_msg, result.next)

def parse_log = LogParser.apply(_)

import LogParser.{ ParseResult, Success, Failure }


defined function filter
defined function parse_log

## The actual questions

For the following problems, you'll find this info useful:

We defined a parse function above, with the following type:
```scala
def parse_log(input: String): ParseResult[Log]
```

A `ParseResult` has 2 cases that we care about (both with a parameter that doesn't matter)
```scala
sealed trait ParseResult
case class Success(result: Log, ignore_this)
case class Failure(msg: String, ignore_this)
```

In [7]:
// Try to parse the logs! Turn the list of strings into a list of ParseResults
val parse_results: List[ParseResult[Log]] =
    logs.map(log_str => parse_log(log_str))

assert(parse_results.length == 11)

parse_results: List[ParseResult[Log]] = List(
  Success(Log(ERROR, 34, "Your toast is burning"), CharSequenceReader()),
  Success(
    Log(WARNING, 30, "No need for a good error message, this'll never break!"),
    CharSequenceReader()
  ),
  Success(
    Log(WARNING, 31, "Stack overflow is down, abandon ship!"),
    CharSequenceReader()
  ),
  Success(
    Log(ERROR, 26, "'cpu' not found. To continue, please run `apt install cpu`"),
    CharSequenceReader()
  ),
  Success(Log(ERROR, 32, "Perfectly balanced..."), CharSequenceReader()),
  Success(Log(WARNING, 33, "... as all things should be"), CharSequenceReader()),
  Success(
    Log(
      ERROR,
      28,
      "\"'You miss 100% of the shots you don't take -Wayne Gretsky' -Michael Scott\" -PPL"
    ),
    CharSequenceReader()
  ),
  Failure("'(' expected but 'R' found", CharSequenceReader('R', ...)),
  Success(
    Log(ERROR, 27, "The real slim shady didn't stand up"),
    CharSequenceReader()
  ),
  Success(
    Log(ERROR, 5059, "S

In [8]:
// Make a list of all the successfully parsed logs
val successes: List[Log] = 
    parse_results
        .filter(result => result match {
            case Success(_, _) => true
            case _ => false
        })
        .map(result => result match {
            case Success(log, _) => log
        })

assert(successes.length == 10)

successes: List[Log] = List(
  Log(ERROR, 34, "Your toast is burning"),
  Log(WARNING, 30, "No need for a good error message, this'll never break!"),
  Log(WARNING, 31, "Stack overflow is down, abandon ship!"),
  Log(ERROR, 26, "'cpu' not found. To continue, please run `apt install cpu`"),
  Log(ERROR, 32, "Perfectly balanced..."),
  Log(WARNING, 33, "... as all things should be"),
  Log(
    ERROR,
    28,
    "\"'You miss 100% of the shots you don't take -Wayne Gretsky' -Michael Scott\" -PPL"
  ),
  Log(ERROR, 27, "The real slim shady didn't stand up"),
  Log(ERROR, 5059, "Send me $100 in bitcoin - from your boss"),
  Log(WARNING, 29, "Your computer is about to explode")
)

In [9]:
// Use the last list to make a new list that only includes 
// ERROR level logs made at or before time 30
val important_logs = successes.filter(log => log match {
    case Log(ERROR, t, _) if t <= 30 => true
    case _ => false
})

assert(important_logs.length == 3)

important_logs: List[Log] = List(
  Log(ERROR, 26, "'cpu' not found. To continue, please run `apt install cpu`"),
  Log(
    ERROR,
    28,
    "\"'You miss 100% of the shots you don't take -Wayne Gretsky' -Michael Scott\" -PPL"
  ),
  Log(ERROR, 27, "The real slim shady didn't stand up")
)

In [10]:
// Compile a report that lists every error message from important_logs
// in one String along with a line number for each seperated by \n

def format_report(msg_so_far: String, line_num: Int, new_msg: String): String = s"$msg_so_far${line_num}. $new_msg\n"

val (report, _) = important_logs.foldLeft(("Report:\n", 1))((acc, log) => {
    val (msg_so_far, line_num) = acc
    val Log(_, _, msg) = log
    (format_report(msg_so_far, line_num, msg), line_num + 1)
})

assert(report =="""Report:
1. 'cpu' not found. To continue, please run `apt install cpu`
2. "'You miss 100% of the shots you don't take -Wayne Gretsky' -Michael Scott" -PPL
3. The real slim shady didn't stand up
""")



defined function format_report
report: String = """Report:
1. 'cpu' not found. To continue, please run `apt install cpu`
2. "'You miss 100% of the shots you don't take -Wayne Gretsky' -Michael Scott" -PPL
3. The real slim shady didn't stand up
"""

## OPTIONAL Exercise: HOFs on non-lists

The following are to help you understand the concepts, you do not have to fully understand the following examples, but they show why these functions are more interesting than just getting rid of a loop.

`map`, `filter`, and the like are important to learn because they are reusable. Use these functions to accomplish the following tasks on non-lists.

`filter` for ParseResult (a failure is treated like an empty list, a success is treated like a one element list). Note the extra argument due to the addition of a message in the "empty" case:
```scala
def filter(result: ParseResult[Log], f: Log => Boolean, error_msg: String): ParseResult[Log]
```

In [11]:
// We've been hacked! Update the parser to only succeed if ...
// ... the time is valid (below 100)
def parse_valid_logs(input: String): ParseResult[Log] = 
    filter(parse_log(input), l => l.time < 100, "We've been hacked!")

defined function parse_valid_logs

Scala has a `Future` type, which represents an asynchronous computation. A `Future[Int]` doesn't contain an int, it represents a long computation in another thread that will eventually have an int.

In [12]:
// Setup for later. You can ignore this.
// This will download stuff the first time you run it

import $ivy.`com.lihaoyi::requests:0.1.4`
import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global

def http_get(url: String): Future[String] = Future(requests.get(url)).map(_.text)

import $ivy.$                            

import scala.concurrent.Future

import scala.concurrent.ExecutionContext.Implicits.global


defined function http_get

In [13]:
val future: Future[String] = http_get("http://www.mocky.io/v2/5c6f9f8d340000745f893183?mocky-delay=3000ms")

// Cast the result to an int and add 5 (all asynchronously)
future.map(text => text.toInt + 5)

future: Future[String] = Success("3")
res12_1: Future[Int] = Success(8)

In [14]:
// One last example, with a new higher order function
// Combine results from two futures:

val shakira1 = http_get("http://www.mocky.io/v2/5c6fa3fc3400004e5f8931a6")
val shakira2 = http_get("http://www.mocky.io/v2/5c6fa47c3400004e5f8931ab")

shakira1.flatMap(text1 => shakira2.map(text2 => text1 + " " + text2))

shakira1: Future[String] = Success("Hips don't")
shakira2: Future[String] = Success("lie.")
res13_2: Future[String] = Success("Hips don't lie.")